<a href="https://colab.research.google.com/github/miotomita/vdata-globalization/blob/main/imf_dots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMF web service (documentation):<br>
https://datahelp.imf.org/knowledgebase/articles/667681-using-json-restful-web-service<br>
IMF DOTS:<br>
https://data.imf.org/?sk=9D6028D4-F14A-464C-A2F2-59B2CD424B85

In [1]:
#!pip install requests
#!pip install pandas
!pip install xlrd -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import requests
import json
import pandas as pd
import numpy as np
import time
import itertools

In [3]:
pd.set_option("display.precision", 8)

# data structures

In [4]:
id = 'DOT'

In [5]:
#data structure
url = f'http://dataservices.imf.org/REST/SDMX_JSON.svc/DataStructure/{id}'
r = requests.get(url)

structure = r.json()

In [6]:
codes = pd.DataFrame(structure['Structure']['CodeLists']['CodeList'])
codes

,@id,@agencyID,@version,@isFinal,@xmlns,Name,Code,Description
0,CL_UNIT_MULT,IMF,1.0,true,http://www.SDMX.org/resources/SDMXML/schemas/v...,"{'@xml:lang': 'en', '#text': 'Scale'}","[{'@value': '0', 'Description': {'@xml:lang': ...",NaN
1,CL_FREQ,IMF,1.0,true,http://www.SDMX.org/resources/SDMXML/schemas/v...,"{'@xml:lang': 'en', '#text': 'Frequency'}","[{'@value': 'A', 'Description': {'@xml:lang': ...","{'@xml:lang': 'en', '#text': 'Frequency'}"
2,CL_AREA_DOT,IMF,1.0,true,http://www.SDMX.org/resources/SDMXML/schemas/v...,"{'@xml:lang': 'en', '#text': 'Geographical Are...","[{'@value': 'AF', 'Description': {'@xml:lang':...",NaN
3,CL_INDICATOR_DOT,IMF,1.0,true,http://www.SDMX.org/resources/SDMXML/schemas/v...,"{'@xml:lang': 'en', '#text': 'Indicator'}","[{'@value': 'TXG_FOB_USD', 'Description': {'@x...",NaN
4,CL_COUNTERPART_AREA_DOT,IMF,1.0,true,http://www.SDMX.org/resources/SDMXML/schemas/v...,"{'@xml:lang': 'en', '#text': 'Counterpart Geog...","[{'@value': 'AF', 'Description': {'@xml:lang':...",NaN
5,CL_TIME_FORMAT,IMF,1.0,true,http://www.SDMX.org/resources/SDMXML/schemas/v...,"{'@xml:lang': 'en', '#text': 'Time format'}","[{'@value': 'P1Y', 'Description': {'@xml:lang'...","{'@xml:lang': 'en', '#text': 'Time formats bas..."


indicators

In [7]:
#indicators
pd.json_normalize(codes.Code[3])

,@value,Description.@xml:lang,Description.#text
0,TXG_FOB_USD,en,"Goods, Value of Exports, Free on board (FOB), ..."
1,TMG_CIF_USD,en,"Goods, Value of Imports, Cost, Insurance, Frei..."
2,TMG_FOB_USD,en,"Goods, Value of Imports, Free on board (FOB), ..."
3,TBG_USD,en,"Goods, Value of Trade Balance, US Dollars"


In [8]:
#unit
pd.json_normalize(codes.Code[0]).head()

,@value,Description.@xml:lang,Description.#text
0,0,en,Units
1,2,en,Hundreds
2,3,en,Thousands
3,6,en,Millions
4,9,en,Billions


In [9]:
#the maximum number of time series that can be returned by CompactData
requests.get('http://dataservices.imf.org/REST/SDMX_JSON.svc/GetMaxSeriesInResult').json()

3000

# areas

In [10]:
#areas
areas = pd.json_normalize(codes.Code[2])
areas.columns = ['code', 'language', 'name']
areas.head()

,code,language,name
0,AF,en,Afghanistan
1,F19,en,Africa not allocated
2,AL,en,Albania
3,DZ,en,Algeria
4,AS,en,American Samoa


In [11]:
#Japanese name(コロナのVdataなどで整理済み、日経表記の日本語名)
meta = pd.read_csv('https://raw.githubusercontent.com/miotomita/meta/main/countries_meta.csv')
meta.loc[meta.code_3digit=='NAM',['code_2digit','ISO2']] = 'NA'

areas['name_ja'] = areas.code.map(meta.set_index('ISO2').name_ja_NK.to_dict())

In [12]:
#lat, long（Google表示）
google= pd.read_html('https://developers.google.com/public-data/docs/canonical/countries_csv')[0]
google = google.rename(columns={'country':'code'})
google.loc[google.name=='Namibia','code'] = 'NA'

areas = areas.merge(google, on='code', how='left',suffixes=('','_google'))

In [13]:
#lat, long (Google表示にない分はmetaから追加=内容確認済み)
null = areas[(~areas.name_ja.isna())&(areas.name_google.isna())].code.to_list()
areas.loc[areas.code.isin(null),'latitude'] = areas.loc[areas.code.isin(null),'code'].map(meta.set_index('ISO2').lat_nikkei.to_dict())
areas.loc[areas.code.isin(null),'longitude'] = areas.loc[areas.code.isin(null),'code'].map(meta.set_index('ISO2').long_nikkei.to_dict())
areas[areas.code.isin(null)]

,code,language,name,name_ja,latitude,longitude,name_google
53,CW,en,Curacao,キュラソー,12.169230,-69.001715,NaN
179,SX,en,Sint Maarten (Dutch part),シント・マールテン,18.039243,-63.059446,NaN
186,SS,en,South Sudan,南スーダン,7.446640,29.831393,NaN


In [14]:
#旧国名
former_countries = pd.DataFrame([
    {'code':'DE2', 'name':'East Germany', 'name_ja':'旧東ドイツ', 'latitude':52.4, 'longitude':12.5}, #https://en.wikipedia.org/wiki/Module:Location_map/data/East_Germany
    {'code':'CSH', 'name':'Former Czechoslovakia', 'name_ja':'旧チェコスロバキア', 'latitude':50.083333, 'longitude':14.416667}, #https://geohack.toolforge.org/geohack.php?pagename=Czechoslovakia&params=50_05_N_14_25_E_region:CZ_type:city
    {'code':'CS', 'name':'Serbia and Montenegro', 'name_ja':'旧セルビア・モンテネグロ', 'latitude':44.816667, 'longitude':20.466667}, #https://geohack.toolforge.org/geohack.php?pagename=Serbia_and_Montenegro&params=44_49_N_20_28_E_source:kolossus-svwiki
    {'code':'SUH', 'name':'Former U.S.S.R.', 'name_ja':'旧ソビエト連邦', 'latitude':55.751244, 'longitude':37.618423}, #Moscow
    {'code':'1C_473', 'name':'Yemen Arab Rep.', 'name_ja':'旧イエメン・アラブ共和国', 'latitude':15.354722, 'longitude':44.206667}, #https://geohack.toolforge.org/geohack.php?pagename=Yemen_Arab_Republic&params=15_21_17_N_44_12_24_E_type:city_source:kolossus-hewiki
    {'code':'1C_459', 'name':'Yemen, P.D. Rep.', 'name_ja':'イエメン人民民主共和国', 'latitude':12.8, 'longitude':45.033}, #https://geohack.toolforge.org/geohack.php?pagename=Democratic_Republic_of_Yemen&params=12.8000_N_45.0330_E_source:wikidata
    {'code':'YUC', 'name':'Former Yugoslavia', 'name_ja':'旧ユーゴスラビア', 'latitude':44.816667, 'longitude':20.45} #https://geohack.toolforge.org/geohack.php?pagename=Yugoslavia&params=44_49_N_20_27_E_type:country_source:kolossus-hewiki
     ])

In [15]:
areas = areas.append(former_countries)
areas = areas[~areas.duplicated(subset='code',keep='last')]

In [16]:
areas['code3'] = areas.code.map(meta.set_index('ISO2').ISO3.to_dict())

In [17]:
na_areas = areas[areas.name_ja.isna()].code.to_list()

In [18]:
areas = areas.drop(['language','name_google'],axis=1)

In [19]:
areas[(~areas.name_ja.isna())&(areas.code3.isna())]

,code,name,name_ja,latitude,longitude,code3
0,DE2,East Germany,旧東ドイツ,52.400000,12.500000,NaN
1,CSH,Former Czechoslovakia,旧チェコスロバキア,50.083333,14.416667,NaN
2,CS,Serbia and Montenegro,旧セルビア・モンテネグロ,44.816667,20.466667,NaN
3,SUH,Former U.S.S.R.,旧ソビエト連邦,55.751244,37.618423,NaN
4,1C_473,Yemen Arab Rep.,旧イエメン・アラブ共和国,15.354722,44.206667,NaN
5,1C_459,"Yemen, P.D. Rep.",イエメン人民民主共和国,12.800000,45.033000,NaN
6,YUC,Former Yugoslavia,旧ユーゴスラビア,44.816667,20.450000,NaN


In [20]:
areas.to_csv('areas.csv', index=False)

In [21]:
areas_r = areas.loc[~areas.latitude.isna()]
areas_r.to_csv('areas_r.csv', index=False)

#data

retrieve data

In [22]:
imf_data = []
indicators = ['TXG_FOB_USD','TMG_FOB_USD','TMG_CIF_USD']

for iter in itertools.product(areas.code, indicators):
  #url
  country_code = iter[0]
  indicator= iter[1]
  id = 'DOT'
  freq = 'A'
  start = 1940
  end = 2022

  url = f'http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/{id}/{freq}.{country_code}.{indicator}.?startPeriod={start}&endPeriod={end}'

  try:
    #get data
    r = requests.get(url)
    data = r.json()
    series = data['CompactData']['DataSet']['Series']

    #cleaning data
    for item in series:
      item['Obs'] = np.nan if 'Obs' not in item.keys() else item['Obs']
      item['Obs'] = [item['Obs']] if type(item['Obs']) != list else item['Obs']

    #append data
    imf_data = imf_data + series

  except:
    pass
  time.sleep(1)

df

In [23]:
#df
df = pd.json_normalize(imf_data, 'Obs', ['@REF_AREA','@COUNTERPART_AREA','@INDICATOR', '@UNIT_MULT'])
df = df.rename(columns = {'@TIME_PERIOD':'year', '@OBS_VALUE':'value', '@OBS_STATUS':'status', '@REF_AREA':'code', '@COUNTERPART_AREA':'counterpart_code', '@INDICATOR':'indicator', '@UNIT_MULT':'unit'})
df = df.drop(['status'], axis=1)

In [24]:
df['name'] = df.code.map(areas.set_index('code').name.to_dict())
df['name_ja'] = df.code.map(areas.set_index('code').name_ja.to_dict())
df['counterpart_name'] = df.counterpart_code.map(areas.set_index('code').name.to_dict())
df['counterpart_name_ja'] = df.counterpart_code.map(areas.set_index('code').name_ja.to_dict())

In [25]:
df['indicator_name'] = df.indicator.map({
    'TXG_FOB_USD':'Goods, Value of Exports, Free on board (FOB), USD',
    'TMG_FOB_USD':'Goods, Value of Imports, Free on board (FOB), USD',
    'TMG_CIF_USD':'Goods, Value of Imports, Cost, insurance, and freight (CIF), USD'
    })

In [26]:
df.value = df.value.astype(float)
df.year = df.year.astype(int)

In [73]:
#すべてmillions USD
df.unit.unique()

array(['6'], dtype=object)

In [75]:
df = df.drop('unit', axis=1)

加工前アウトプット

In [76]:
df.to_csv('imf_dots_all.csv',index=False)

In [77]:
country_df = df[df.code.isin(areas_r.code)]
country_df.to_csv('imf_dots_all_countries.csv', index=False)

In [125]:
df.loc[(df.code=='W00')&(df.counterpart_code=='W00')].to_csv('imf_dots_world.csv',index=False)

輸出入、貿易総額

In [116]:
#export
exports = df[df.indicator=='TXG_FOB_USD']
exports.to_csv('imf_dots_exports.csv',index=False)

In [117]:
#import
df[df.indicator=='TMG_FOB_USD'].to_csv('imf_dots_imports_FOB.csv',index=False)
df[df.indicator=='TMG_CIF_USD'].to_csv('imf_dots_imports_CIF.csv',index=False)

In [118]:
#importのFOBデータがある国
df[df.indicator=='TMG_FOB_USD'].name.unique()

array(['Australia', 'Bermuda', 'Brazil', 'Canada', 'Dominican Republic',
       'Mexico', 'Papua New Guinea', 'Paraguay', 'Peru', 'Romania',
       'Solomon Islands', 'South Africa',
       'South African Common Customs Area (SACCA)', 'Zimbabwe',
       'Former Czechoslovakia'], dtype=object)

In [120]:
#importデータはFOBがあるものはFOB, CIFのみはCIFに
imports= df[df.indicator!='TXG_FOB_USD'].sort_values(by='indicator')
imports = imports[~imports.duplicated(subset=['year','code','counterpart_code'], keep='first')]

In [121]:
#貿易総額
total_trade = pd.concat([imports, exports]).groupby(['year', 'code', 'counterpart_code', 'name','name_ja', 'counterpart_name', 'counterpart_name_ja']).value.sum().reset_index()

In [122]:
#地域データなど除外して保存
imports[imports.code.isin(areas_r.code)].to_csv('imf_dots_imports_countries.csv',index=False)
exports[exports.code.isin(areas_r.code)].to_csv('imf_dots_exports_countries.csv',index=False)
total_trade[total_trade.code.isin(areas_r.code)].to_csv('imf_dots_totaltrade_countries.csv',index=False)

確認

In [123]:
#データのある国の数
df[~df.duplicated(subset=['year','code'],keep='first')].year.value_counts().sort_index()

1948    100
1949     93
1950     93
1951     96
1952     96
       ... 
2017    227
2018    227
2019    227
2020    227
2021    227
Name: year, Length: 74, dtype: int64

In [124]:
##Japanは1948~
#Chinaは1961~
df[(~df.sort_values(by='year').duplicated(subset='code', keep='first'))&(df.code.isin(['JP','US','CN','GB','DE','CA','SG']))&(df.indicator=='TXG_FOB_USD')]

,year,value,code,counterpart_code,indicator,name,name_ja,counterpart_name,counterpart_name_ja,indicator_name
400884,1948,5.00,CA,IL,TXG_FOB_USD,Canada,カナダ,Israel,イスラエル,"Goods, Value of Exports, Free on board (FOB), USD"
497932,1961,0.60,CN,DZ,TXG_FOB_USD,China,中国,Algeria,アルジェリア,"Goods, Value of Exports, Free on board (FOB), USD"
889469,1948,1.00,DE,AU,TXG_FOB_USD,Germany,ドイツ,Australia,オーストラリア,"Goods, Value of Exports, Free on board (FOB), USD"
2013384,1960,0.73,SG,MM,TXG_FOB_USD,Singapore,シンガポール,Myanmar,ミャンマー,"Goods, Value of Exports, Free on board (FOB), USD"
2394571,1948,409.30,GB,F97,TXG_FOB_USD,United Kingdom,英国,Middle East,NaN,"Goods, Value of Exports, Free on board (FOB), USD"


Flourish用

In [131]:
#file size mitigation for Flourish
exports_r = exports[exports.code.isin(areas_r.code)].drop(['indicator','indicator_name','name','name_ja','counterpart_name','counterpart_name_ja'], axis=1)
exports_r.value = exports_r.value.round(4)
exports_r.to_csv('imf_dots_countries_export_r.csv', index=False)

In [132]:
#rankづけ
exports_r['rank'] = exports_r.groupby('year').value.rank(ascending=False, method='first')

In [133]:
exports_r[(exports_r['rank']<1000)].to_csv('imf_dots_countries_export_top1000.csv', index=False)

#tug-of-war

In [136]:
#米中が相手国の貿易総額のみ抜粋
tug = total_trade[(total_trade.code.isin(areas_r.code))&(total_trade.counterpart_code.isin(['US','CN']))]

In [142]:
tug = tug.pivot(index=['year','code','name_ja'],columns='counterpart_code',values='value')

In [144]:
tug.head()

counterpart_code         CN      US
year code name_ja                  
1948 AN   オランダ領アンティル   0.11  127.03
     AO   アンゴラ          NaN   13.20
     AR   アルゼンチン        NaN  763.10
     AT   オーストリア       0.80  282.50
     AU   オーストラリア     16.22  268.79

In [149]:
tug['CN_ratio'] = (tug.CN / tug[['CN','US']].fillna(0).sum(axis=1)) *100
tug['US_ratio'] = (tug.US / tug[['CN','US']].fillna(0).sum(axis=1)) *100

In [150]:
tug.loc[(2021, 'JP','日本')]

counterpart_code
CN          348844.56412500
US          218438.52267500
CN_ratio        61.49391234
US_ratio        38.50608766
Name: (2021, JP, 日本), dtype: float64

In [151]:
tug.loc[(2021, 'DE','ドイツ')]

counterpart_code
CN          239682.33451600
US          208594.64460600
CN_ratio        53.46746446
US_ratio        46.53253554
Name: (2021, DE, ドイツ), dtype: float64

In [154]:
tug = tug.reset_index().rename_axis('',axis=1)
tug['code3'] = tug.code.map(areas.set_index('code').code3.to_dict())
#tug = tug.set_index(['code3','year'])

In [155]:
tug.head()

,year,code,name_ja,CN,US,CN_ratio,US_ratio,code3
0,1948,AN,オランダ領アンティル,0.11,127.03,0.08651880,99.91348120,ANT
1,1948,AO,アンゴラ,NaN,13.20,NaN,100.00000000,AGO
2,1948,AR,アルゼンチン,NaN,763.10,NaN,100.00000000,ARG
3,1948,AT,オーストリア,0.80,282.50,0.28238616,99.71761384,AUT
4,1948,AU,オーストラリア,16.22,268.79,5.69102838,94.30897162,AUS


GDPデータを追加<br>
①IMF

In [156]:
#source 1: WEO
url = 'https://www.imf.org/-/media/Files/Publications/WEO/WEO-Database/2022/WEOOct2022all.ashx'
r = requests.get(url)

with open('weo2022.xls', 'wb') as f:
    f.write(r.content)

weo = pd.read_csv('weo2022.xls', sep='\t', thousands=',', na_values=['n/a', '--'], encoding='windows-1252') 

In [157]:
#nominal GDP
weo_ngdp = weo[weo['WEO Subject Code']=='NGDPD'].rename(columns ={'ISO':'code3'} )
weo_ngdp = weo_ngdp.set_index('code3').loc[:,'1980':'2022'].rename_axis('year', axis=1).unstack().rename('NGDP').reset_index()
weo_ngdp.year = weo_ngdp.year.astype(int)
weo_ngdp.NGDP = weo_ngdp.NGDP.astype(float) *1000   #unit USD billions-->USD millions

In [158]:
tug = tug.merge(weo_ngdp, on=['year','code3'], how='left')

②UN（旧ソ連などformer countries用）<br>
https://unstats.un.org/unsd/snaama/downloads

In [159]:
#GDP and its breakdown at current prices in US Dollars
url = 'https://unstats.un.org/unsd/amaapi/api/file/2'
r = requests.get(url)
with open('un_current.xls', 'wb') as f:
    f.write(r.content)

un = pd.read_excel('un_current.xls', header=2)

In [160]:
#GDP and its breakdown at constant 2015 prices in US Dollars
#url = 'https://unstats.un.org/unsd/amaapi/api/file/6'

In [161]:
#unit = USD
un[(un.IndicatorName=='Gross Domestic Product (GDP)')&(un.Country=='Japan')][2020] /1000000000000

1676    5.05775896
Name: 2020, dtype: float64

In [162]:
un = un[(un.IndicatorName=='Gross Domestic Product (GDP)')].set_index('Country')

In [163]:
un = un.loc[:,1970:].rename_axis('year',axis=1).unstack().rename('UNGDP').reset_index()

In [164]:
#unit = millionに変更
un.UNGDP = un.UNGDP.astype(float) / 1000000

In [165]:
#former countries
un.loc[un.Country=='USSR (Former)','code'] = 'SUH'
un.loc[un.Country=='Czechoslovakia (Former)','code'] = 'CSH'
un.loc[un.Country=='Yemen Arab Republic (Former)','code'] = '1C_473'
un.loc[un.Country=='Yemen Democratic (Former)','code'] = '1C_459'
un.loc[un.Country=='Yugoslavia (Former)','code'] = 'YUC'

#no data available
#un[un.Country.str.contains('Germany', case=False)].Country.unique()
#un.loc[un.Country.str.contains('Serbia', case=False)].Country.unique()

In [166]:
un_former = un.loc[~un.code.isna(),['code','year','UNGDP']]

In [167]:
tug = tug.merge(un_former,on=['code','year'],how='left')

In [168]:
tug['GDP'] = tug[['NGDP','UNGDP']].ffill(axis=1).UNGDP

米中間の表示位置

In [169]:
#flourishの表示用
tug['ratio'] = 100 - tug.US_ratio
tug['ratio'] = tug[['CN_ratio','ratio']].ffill(axis=1).ratio

#USとCN本体の位置を調整（仮置き）
tug.loc[tug.code=='US','ratio'] = -20
tug.loc[tug.code=='CN','ratio'] = 120

In [170]:
tug[(tug.CN.isna())&(tug.US.isna())&(tug.ratio.isin([0,100]))]

,year,code,name_ja,CN,US,CN_ratio,US_ratio,code3,NGDP,UNGDP,GDP,ratio


image(country flags)

In [171]:
tug['image'] = tug.code.apply(lambda x: f'https://hatscripts.github.io/circle-flags/flags/{x.lower()}.svg')

In [172]:
#image not available
null = ['AN', 'CSH', 'YUC', 'DE2', 'SUH', '1C_459', '1C_473', 'CS']
tug.loc[tug.code.isin(null),'image'] = np.nan

In [173]:
tug.to_csv('tug_o_war.csv',index=False)

In [174]:
#プレゼン用のデータ仮出し
for year in list(np.arange(1950,2022,5)) + [2021]:
  filename = f'tug_o_war_{year}.csv'
  tug[tug.year==year].to_csv(filename)

In [175]:
un[un.Country.str.contains('China', case=False)]

,year,Country,UNGDP,code
40,1970,China,9.26026178e+04,NaN
41,1970,"China, Hong Kong SAR",3.81249381e+03,NaN
42,1970,"China, Macao SAR",1.69026522e+02,NaN
260,1971,China,9.98005753e+04,NaN
261,1971,"China, Hong Kong SAR",4.46139945e+03,NaN
...,...,...,...,...
10821,2019,"China, Hong Kong SAR",3.65708203e+05,NaN
10822,2019,"China, Macao SAR",5.51539107e+04,NaN
11040,2020,China,1.47228005e+07,NaN
11041,2020,"China, Hong Kong SAR",3.49444713e+05,NaN


In [176]:
un.to_csv('un_gdp_USD_millions.csv')

In [177]:
weo.to_csv('weo_nominal_gdp_USD_millions.csv')

In [178]:
counterparts = df[df.indicator=='TXG_FOB_USD'].groupby(['year','name']).counterpart_name.count().reset_index().pivot(index='name',columns='year', values='counterpart_name')

In [ ]:
#counterparts.to_excel('counterparts.xls')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  """Entry point for launching an IPython kernel.
